In [1]:
import pandas as pd

# Load the dataframes
mod_df = pd.read_excel("mod_df_gb.xlsx")
ad_df = pd.read_excel("ad_df_gb.xlsx")

# Sort both dataframes by their respective scores in descending order
sorted_mod_df = mod_df.sort_values(by="mod_score_pred", ascending=False)
sorted_ad_df = ad_df.sort_values(by="ad_score_pred", ascending=False)

# Initial matching based on delivery country
matches = []
available_mods = sorted_mod_df.copy()
for _, ad_row in sorted_ad_df.iterrows():
    country = ad_row['delivery_country']
    suitable_mod = available_mods[available_mods[f'"{country}"'] == 1.0].head(1)
    if not suitable_mod.empty:
        matches.append((ad_row['ad_id'], suitable_mod['moderator'].iloc[0]))
        available_mods.drop(suitable_mod.index, inplace=True)
matched_df = pd.DataFrame(matches, columns=['ad_id', 'moderator'])

# Re-use moderators for unmatched ads
additional_matches = []
unmatched_ads = sorted_ad_df[~sorted_ad_df['ad_id'].isin(matched_df['ad_id'])]
for _, ad_row in unmatched_ads.iterrows():
    country = ad_row['delivery_country']
    suitable_mod = sorted_mod_df[sorted_mod_df[f'"{country}"'] == 1.0].head(1)
    if not suitable_mod.empty:
        additional_matches.append((ad_row['ad_id'], suitable_mod['moderator'].iloc[0]))
additional_matched_df = pd.DataFrame(additional_matches, columns=['ad_id', 'moderator'])
combined_matched_df = pd.concat([matched_df, additional_matched_df])

# Language-based matching for remaining unmatched ads
country_to_language = {
    'US': 'English', 'CA': 'French', 'GB': 'English', 'AU': 'English', 'NZ': 'English', 'IE': 'English', 'ZA': 'English', 'IN': 'English', 'PH': 'English', 'SG': 'Chinese', 'MY': 'English', 'BR': 'Portuguese', 'PT': 'Portuguese', 'ES': 'Spanish', 'MX': 'Spanish', 'AR': 'Spanish', 'PE': 'Spanish', 'VE': 'Spanish', 'CL': 'Spanish', 'EC': 'Spanish', 'CO': 'Spanish', 'UY': 'Spanish', 'PA': 'Spanish', 'CR': 'Spanish', 'CU': 'Spanish', 'FR': 'French', 'BE': 'Dutch', 'CH': 'German', 'DE': 'German', 'AT': 'German', 'IT': 'Italian', 'JP': 'Japanese', 'KR': 'Korean', 'CN': 'Chinese', 'TW': 'Chinese', 'HK': 'Chinese', 'RU': 'Russian', 'NL': 'Dutch', 'SE': 'Swedish', 'NO': 'Norwegian', 'DK': 'Danish', 'FI': 'Finnish', 'PL': 'Polish', 'CZ': 'Czech', 'GR': 'Greek', 'HU': 'Hungarian', 'RO': 'Romanian', 'TR': 'Turkish', 'TH': 'Thai', 'ID': 'Indonesian'
}

additional_language_matches = []
unmatched_ads_with_no_mods = sorted_ad_df[~sorted_ad_df['ad_id'].isin(combined_matched_df['ad_id'])]
for _, ad_row in unmatched_ads_with_no_mods.iterrows():
    country = ad_row['delivery_country']
    language = country_to_language.get(country, None)
    if language:
        matching_countries = [f'"{k}"' for k, v in country_to_language.items() if v == language and f'"{k}"' in sorted_mod_df.columns]
        if matching_countries:
            suitable_mod = sorted_mod_df[sorted_mod_df[matching_countries].sum(axis=1) > 0].head(1)
            if not suitable_mod.empty:
                additional_language_matches.append((ad_row['ad_id'], suitable_mod['moderator'].iloc[0]))
additional_language_matched_df = pd.DataFrame(additional_language_matches, columns=['ad_id', 'moderator'])
final_matched_df = pd.concat([combined_matched_df, additional_language_matched_df])


final_matched_df.to_excel("final_matched_pairs.xlsx", index=False)

KeyError: '"CA"'